# Conv1d + Conv2d
----

trello: https://trello.com/c/CDnYK17O


front-end를 conv1d로 두고

뒤 layer를 conv2d로 두었을 떄 효과에 대해 ... 알아보기 위하여..

----

일단 성능이 좋게 나오지않는다..

## Environment

In [1]:
%load_ext autoreload
%autoreload 2
%pylab
%matplotlib inline

import pandas as pd
import pickle
import numpy as np
import sys
import os

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
sys.path.append('../')
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

### Configuration

In [3]:
from sv_system.utils.parser import set_train_config
import easydict

# datasets
# voxc1_fbank_xvector
# gcommand_fbank_xvector

args = easydict.EasyDict(dict(dataset="voxc1_fbank_xvector",
                              input_frames=500, splice_frames=[300, 500], stride_frames=1, input_format='fbank',
                              cuda=True,
                              lrs=[0.1, 0.01], lr_schedule=[20], seed=1337,
                              no_eer=False,
                              batch_size=128,
                              arch="tdnn_conv", loss="softmax",
                              n_epochs=50
                             ))
config = set_train_config(args)

### Dataset and Dataloader

In [4]:
from sv_system.data.data_utils import find_dataset, find_trial

_, datasets = find_dataset(config, basedir='../')
trial = find_trial(config, basedir='../')

In [5]:
from sv_system.data.dataloader import init_loaders

dataloaders = init_loaders(config, datasets)

### Define Model

In [6]:
# from sv_system.model.model_utils import find_model
# model = find_model(config)

In [15]:
from sv_system.model.tdnnModel import gTDNN
from sv_system.model.ResNet34 import BasicBlock

import torch.nn as nn
import torch.nn.functional as F

class tdnn_conv(nn.Module):
    """xvector architecture"""
    def __init__(self, config, n_labels=31):
        super(tdnn_conv, self).__init__()
        inDim = config['input_dim']
        self.tdnn = nn.Sequential(
            nn.Conv1d(inDim, 256, stride=1, dilation=1, kernel_size=5),
            nn.BatchNorm1d(256),
            nn.ReLU(True),
        )

        layers = [3,4,6,3]
        self.inplanes = inplanes = 4
        self.extractor = nn.Sequential(
            nn.Conv2d(1, inplanes, kernel_size=3, stride=1, padding=3,
                                   bias=False),
            nn.BatchNorm2d(inplanes),
            nn.ReLU(inplace=True),
            self._make_layer(BasicBlock, inplanes, layers[0]),
            self._make_layer(BasicBlock, 2*inplanes, layers[1], stride=2),
            self._make_layer(BasicBlock, 4*inplanes, layers[2], stride=2),
            self._make_layer(BasicBlock, 8*inplanes, layers[3], stride=2)
        )

        self.classifier = nn.Linear(8*inplanes, n_labels)

        self._init_weight()

    def _init_weight(self):
        for m in self.modules():
            # classifier does not contain Conv2d or BN2d
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.Conv1d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def embed(self, x):
        x = x.squeeze()
        # (batch, time, freq) -> (batch, freq, time)
        x = x.permute(0,2,1)
        x = self.tdnn(x)
        x = x.unsqueeze(1)
#         print(x.shape)
        x = self.extractor(x)
        x = F.avg_pool2d(x,x.shape[-2:])
        x = x.view(x.size(0), -1)

        return x

    def forward(self, x):
        x = self.embed(x)
        x = self.classifier(x)

        return x

In [16]:
model = tdnn_conv(config, config['n_labels'])

In [17]:
if not config['no_cuda']:
    model = model.cuda()

### Model Train

In [19]:
from sv_system.train.train_utils import set_seed, find_optimizer
from torch.optim.lr_scheduler import ReduceLROnPlateau

criterion, optimizer = find_optimizer(config, model)
scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.1, patience=5)

In [20]:
set_seed(config)

In [21]:
if not config['no_eer']:
    train_loader, val_loader, test_loader, sv_loader = dataloaders
else:
    train_loader, val_loader, test_loader = dataloaders

In [22]:
import torch

def train(model):
    model.train()
    loss_sum = 0
    n_corrects = 0
    total = 0
    for batch_idx, (X, y) in enumerate(train_loader):
        if not config['no_cuda']:
            X = X.cuda()
            y = y.cuda()

        optimizer.zero_grad()

        logit = model(X)
        loss = criterion(logit, y)
        loss.backward()
        optimizer.step()
                        
        loss_sum += loss.item()
        n_corrects += torch.sum(torch.eq(torch.argmax(logit, dim=1), y)).item()
        total += y.size(0)
        
        if (batch_idx+1) % 100 == 0:
            print("Batch {}/{}\t Loss {:.6f}" \
                  .format(batch_idx+1, len(train_loader), loss_sum /(batch_idx+1),)
                 )
        acc = n_corrects / total

    acc = n_corrects / total
    return loss_sum, acc 

In [23]:
from sv_system.train.si_train import val, sv_test

for epoch_idx in range(0, config['n_epochs']):
    print("-"*30)
    curr_lr = optimizer.state_dict()['param_groups'][0]['lr']
    print("curr_lr: {}".format(curr_lr))

#     train code
    train_loss, train_acc = train(model)

#     validation code
    val_loss, val_acc = val(config, val_loader, model, criterion)
    print("epoch #{}, val accuracy: {}".format(epoch_idx, val_acc))

#     evaluate best_metric
    if not config['no_eer']:
        # eer validation code
        eer, label, score = sv_test(config, sv_loader, model, trial)
        print("epoch #{}, sv eer: {}".format(epoch_idx, eer))
    
    scheduler.step(train_loss)

------------------------------
curr_lr: 0.1
Batch 100/1042	 Loss 6.991380
Batch 200/1042	 Loss 6.906220
Batch 300/1042	 Loss 6.825552
Batch 400/1042	 Loss 6.724923
Batch 500/1042	 Loss 6.615981
Batch 600/1042	 Loss 6.513283
Batch 700/1042	 Loss 6.414030
Batch 800/1042	 Loss 6.322789
Batch 900/1042	 Loss 6.237069
Batch 1000/1042	 Loss 6.155338
epoch #0, val accuracy: 0.009139151312410831
epoch #0, sv eer: 0.20950910446171866
------------------------------
curr_lr: 0.1
Batch 100/1042	 Loss 5.257168
Batch 200/1042	 Loss 5.223507
Batch 300/1042	 Loss 5.187330
Batch 400/1042	 Loss 5.149717
Batch 500/1042	 Loss 5.108807
Batch 600/1042	 Loss 5.071156
Batch 700/1042	 Loss 5.035321
Batch 800/1042	 Loss 5.001936
Batch 900/1042	 Loss 4.966598
Batch 1000/1042	 Loss 4.929929
epoch #1, val accuracy: 0.021816037595272064
epoch #1, sv eer: 0.18187626450857203
------------------------------
curr_lr: 0.1
Batch 100/1042	 Loss 4.422289
Batch 200/1042	 Loss 4.382648
Batch 300/1042	 Loss 4.338158
Batch 400/

Batch 200/1042	 Loss 1.411238
Batch 300/1042	 Loss 1.414634
Batch 400/1042	 Loss 1.420889
Batch 500/1042	 Loss 1.430446
Batch 600/1042	 Loss 1.435078
Batch 700/1042	 Loss 1.439196
Batch 800/1042	 Loss 1.444803
Batch 900/1042	 Loss 1.455286
Batch 1000/1042	 Loss 1.458860
epoch #19, val accuracy: 0.21104277670383453
epoch #19, sv eer: 0.1481205409434565
------------------------------
curr_lr: 0.1
Batch 100/1042	 Loss 1.376482
Batch 200/1042	 Loss 1.374528
Batch 300/1042	 Loss 1.377743
Batch 400/1042	 Loss 1.394299
Batch 500/1042	 Loss 1.397961
Batch 600/1042	 Loss 1.409074
Batch 700/1042	 Loss 1.416968
Batch 800/1042	 Loss 1.421832
Batch 900/1042	 Loss 1.425110
Batch 1000/1042	 Loss 1.431569
epoch #20, val accuracy: 0.22706788778305054
epoch #20, sv eer: 0.15323181769779576
------------------------------
curr_lr: 0.1
Batch 100/1042	 Loss 1.326941
Batch 200/1042	 Loss 1.356696
Batch 300/1042	 Loss 1.376526
Batch 400/1042	 Loss 1.383783
Batch 500/1042	 Loss 1.387747
Batch 600/1042	 Loss 1.

Batch 400/1042	 Loss 1.180897
Batch 500/1042	 Loss 1.194386
Batch 600/1042	 Loss 1.201495
Batch 700/1042	 Loss 1.208444
Batch 800/1042	 Loss 1.216177
Batch 900/1042	 Loss 1.222077
Batch 1000/1042	 Loss 1.226234
epoch #38, val accuracy: 0.14260445535182953
epoch #38, sv eer: 0.15903524651261847
------------------------------
curr_lr: 0.1
Batch 100/1042	 Loss 1.167061
Batch 200/1042	 Loss 1.163484
Batch 300/1042	 Loss 1.168688
Batch 400/1042	 Loss 1.181935
Batch 500/1042	 Loss 1.193815
Batch 600/1042	 Loss 1.199134
Batch 700/1042	 Loss 1.206444
Batch 800/1042	 Loss 1.209265
Batch 900/1042	 Loss 1.212929
Batch 1000/1042	 Loss 1.218889
epoch #39, val accuracy: 0.18280407786369324
epoch #39, sv eer: 0.14391438611436483
------------------------------
curr_lr: 0.1
Batch 100/1042	 Loss 1.150017
Batch 200/1042	 Loss 1.149382
Batch 300/1042	 Loss 1.169692
Batch 400/1042	 Loss 1.187703
Batch 500/1042	 Loss 1.191653
Batch 600/1042	 Loss 1.199549
Batch 700/1042	 Loss 1.203072
Batch 800/1042	 Loss 1